In [1]:
from utils import *
import warnings
warnings.filterwarnings("ignore")
from shapely import Polygon, box, Point

In [ ]:
bbox = box(-180, -60, 180, 85)
km = 1 / 120

In [4]:
def get_gdf_conflict_1year(year):
    shp_file = path_data / f"PSM/sample_point/conflict_{year}.shp"
    gdf_conflict_1year = gpd.read_file(shp_file)
    return gdf_conflict_1year

In [6]:
def cal_voronoi_ratio(year, radius):
    gdf_conflict_1year = get_gdf_conflict_1year(year)
    gdf_voronoi = gpd.GeoDataFrame({}, geometry=gdf_conflict_1year.voronoi_polygons(extend_to=bbox))

    gdf_voronoi_clip = gdf_voronoi.sjoin(gdf_conflict_1year)\
        .assign(geo_buffer=lambda _df: _df[["x", "y"]].apply(lambda xy: Point(*xy).buffer(radius * km, quad_segs=64), axis=1))\
        .assign(voronoi_clip=lambda _df: _df[["geometry", "geo_buffer"]].apply(lambda g_: g_[0].intersection(g_[1]), axis=1))
    gdf_voronoi_clip = gdf_voronoi_clip.drop(columns=["geometry", "geo_buffer"])\
        .rename(columns={"voronoi_clip": "geometry"}).set_geometry("geometry")\
        .assign(area=lambda _df: _df.area)\
        .assign(ratio=lambda _df: _df["area"] / ((radius * km) **2 * np.pi))
    df_voronoi_ratio = gdf_voronoi_clip[["idx", "ratio"]].assign(year=year)
    return df_voronoi_ratio

In [7]:
def cal_voronoi_ratio_1radius(radius):
    lst_df_voronoi_ratio = []
    for year in range(2002, 2023):
        df_voronoi_ratio = cal_voronoi_ratio(year, radius).sort_values("idx").reset_index(drop=True)
        lst_df_voronoi_ratio.append(df_voronoi_ratio)
    df_voronoi_ratio_m = pd.concat(lst_df_voronoi_ratio).reset_index(drop=True).assign(radius=radius)
    return df_voronoi_ratio_m

In [12]:
rafius = 10
if not (out_file:=path_data / f"PSM/voronoi/{radius}km.csv").exists():
    df_voronoi_ratio_m = cal_voronoi_ratio_1radius(radius)
    df_voronoi_ratio_m.to_csv(out_file)